# Option Pricing Using Amplitude Estimation

This notebook demonstrates how to use the `construct_finance_model` function, which constructs an option pricing model. For more comprehensive explanation on the algorithm see [Option Pricing notebook](https://github.com/Classiq/classiq-library/blob/main/applications/finance/option_pricing/option_pricing.ipynb).


Option pricing deals with evaluating average values $E[f(x)]$, where $f$ is some financial payoff function, and the probability distribution is given. A quantum algorithm for this task is the Amplitude Estimation algorithm, which can estimate the average with a
convergence rate of $\Omega(1/M^{1/2})$, compared to $\Omega(1/M)$ in the classical case, where $M$ is the number of Grover iterations in the quantum case and the number of the Monte Carlo samples in the classical case. This represents a theoretical quadratic speed-up of the quantum method over classical Monte Carlo methods.

## 1. Defining a Financial Function Input

First, we define the probability distribution. The constructor function supports two different built-in functions, a log-normal distribution, or a Gaussian one. In the example below we choose the former:

In [1]:
from classiq.applications.finance import log_normal_model_input

num_qubits = 5
mu = 0.7
sigma = 0.13

log_normal_model = log_normal_model_input.LogNormalModelInput(
    num_qubits=num_qubits, mu=mu, sigma=sigma
)

Next, we define the payoff function. The constructor supports two built-in payoff functions: European Call Option or Value at Risk:

$f_{\rm european}(S)=\
\Bigg\{\begin{array}{lr}
    0, & \text{when } K\geq S\\
    S - K, & \text{when } K < S\end{array},
$

$f_{\rm var}(S)=\
\Bigg\{\begin{array}{lr}
    0, & \text{when } K\geq S\\
    1, & \text{when } K < S\end{array}.
$

This notebook demonstrates for the European Call Option.

In [2]:
from classiq.applications.finance import function_input

threshold = 2

condition = function_input.FunctionCondition(threshold=threshold, larger=True)
finance_function = function_input.FinanceFunctionInput(
    f="european call option",
    condition=condition,
)

## 2. Constructing and Synthesizing a Finance Model

Finally, we construct the model using the `construct_finance_model` function. We pass the `phase_port_size` which will set the accuracy of the calculation.

In [3]:
from classiq import construct_finance_model

qmod = construct_finance_model(
    finance_model_input=log_normal_model,
    finance_function_input=finance_function,
    phase_port_size=1,
)

In [4]:
from classiq import write_qmod

write_qmod(qmod, "option_pricing")

We synthesize and visualize the circuit

In [5]:
from classiq import show, synthesize

qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/a57d2c5e-b2e1-4918-8c4c-7ca30fb27d56?version=0.41.0.dev39%2B79c8fd0855


## 3. Executing to Find the Average Value

In [6]:
from classiq import execute

results = execute(qprog).result()

Printing out the result estimation of the options price :

In [7]:
print(results[1].name, ":", results[1].value)

estimation : 0.3509999999999999
